# Schrodinger Equation Simulation

### 2D Schrodinger Equation

The time dependent Schrodinger equation for the complex wave function $\psi(x,y,t)$, with $h^{2} / 2m = 1$ is

$$
i \frac{\partial \psi(x,y,t)}{\partial t} = -\left(\frac{\partial^{2} \psi(x, y, t)}{\partial x^{2}}+\frac{\partial^{2} \psi(x, y, t)}{\partial y^{2}}\right)+V(x, y) \psi(x, y, t)
$$

where our Hamiltonian assumes the form

$$ 
H = -\nabla^{2} + V(x,y) 
$$

The quantum particle in our simulation is initialized as a guassian wave packet moving with initial momentum ($k_{x0}$, $k_{y0}$) at an initial location ($x_{0}$, $y_{0}$) with an initial width ($a_{x0}$, $a_{y0}$). The initial wave function takes the form

$$ 
\psi(x,y,t=0) = e^{-\frac{1}{2} (\frac{x - x_{0}}{a_{x0}})^{2}} e^{-\frac{1}{2} (\frac{y - y_{0}}{a_{y0}})^{2}} e^{i( k_{x0} x + k_{y0} y)} 
$$

### Alternating Direction Implicit Method 

*Copied from course notes...*

The ADI method is a powerful implementaion of the operator splitting concept that can be used to solve multi dimensional relaxation iterations. In particular, it can be used for multi dimensional parabolic type equations (it is not necessary in the d=1 implementation of the Crank-Nicholson algorithm that we studied, but it would be useful in diffusion in higher dimensions), or in elliptic problems that require an iterative solution.

The method aims to preserve the algorithm simplicity of explicit methods, while achieving the same stability as implicit methods. We illustrate the method with one example here: the two dimensional diffusion equation,

$$
\frac{\partial u}{\partial t} = D \left( \frac{\partial^{2} u}{\partial x^{2}} + \frac{\partial^{2} u}{\partial y^{2}} \right)
$$

The ADI spliting consists of splitting the interval $\Delta t$ in two sub intervals of size $\Delta t/2$, solve the same equation in both, _but use a different algorithm_ in each of the intervals: use an explicit rule along one direction and implicit in the other in one half step, and reverse that in the second half step:

$$
\frac{u_{ij}^{n+1/2} - u_{ij}^{n}}{\Delta t/2} = D \left( \frac{u_{i+1 \ j}^{n+1/2}-2u_{ij}^{n+1/2}+u_{i-1 \ j}^{n+1/2}}{\Delta x^{2}} + \frac{u_{i \ j+1}^{n}-2u_{ij}^{n}+u_{i \ j-1}^{n}}{\Delta ^{2}} \right)
$$
$$
\frac{u_{ij}^{n+1} - u_{ij}^{n+1/2}}{\Delta t/2} = D \left( \frac{u_{i+1 \ j}^{n+1/2}-2u_{ij}^{n+1/2}+u_{i-1 \ j}^{n+1/2}}{\Delta x^{2}} + \frac{u_{i \ j+1}^{n+1}-2u_{ij}^{n+1}+u_{i \ j-1}^{n+1}}{\Delta ^{2}} \right)
$$

with $i = 1, \ldots, N$, $j = 1, \ldots, N$. With this choice of splitting, the first half step is explicit in the $y$ direction but implicit in the $x$ direction, whereas the second half step is explicit in the $x$ direction, but implicit in the $y$ direction. By integrating implicitly only one direction in coordinate space, the iteration is tractable. For example, let us rearrange the first half step in such a way that it makes the iteration more explicit,

$$
\frac{D \Delta t}{\Delta x^{2}} u_{i-1 \ j}^{n+1/2} - 2 \left( 1 + \frac{D \Delta t}{\Delta x^{2}} \right) u_{ij}^{n+1/2} + \frac{D \Delta t}{\Delta x^{2}} u_{i+1 \ j}^{n+1/2} = - \frac{D \Delta t}{\Delta y^{2}} u_{i\ j-1}^{n} - 2 \left( 1 - \frac{D \Delta t}{\Delta y^{2}} \right) u_{ij}^{n} - \frac{D \Delta t}{\Delta y^{2}} u_{i \ j+1}^{n}.
$$

As written, for each $j$ fixed, this is a tridiagonal system of equations for $u_{ij}^{n+1/2}$, where the right hand side is known if the solution at time step $n$, $u_{ij}^{n}$, is known. The solution of the tridiagonal system will give the solution for fixed $j$, so that $N$ such systems need to be solve to find the complete $u_{ij}^{n+1/2}$ for all $j$. Once this is achieved, the solution for the second half step is obtained, in which rows and columns are interchaged,

$$
\frac{D \Delta t}{\Delta y^{2}} u_{i \ j-1}^{n+1} - 2 \left( 1 + \frac{D \Delta t}{\Delta y^{2}} \right) u_{ij}^{n+1} + \frac{D \Delta t}{\Delta y^{2}} u_{i \ j+1}^{n+1} = - \frac{D \Delta t}{\Delta x^{2}} u_{i-1\ j}^{n+1/2} - 2 \left( 1 - \frac{D \Delta t}{\Delta x^{2}} \right) u_{ij}^{n+1/2} - \frac{D \Delta t}{\Delta x^{2}} u_{i+1 \ j}^{n+1/2}.
$$

Now we have $i=1,... N$ systems of equations. Each system is solved for fixed $i$.

This method is very efficient as the number of calculations required for the solution of a tridiagonal system is linear in $N$. Since one solves $N$ equations, the number of computations will scale with $N^{2}$, which is good for a system in two dimensions.

In [1]:
import os
import sys
import PIL
import numpy as np
from PIL import Image

import scipy.linalg
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import csr_matrix

from time import time, sleep
import matplotlib.pyplot as plt
from matplotlib import animation
from colorsys import hls_to_rgb
from numba import njit, prange
from IPython.display import display, clear_output
from mpl_toolkits.mplot3d import Axes3D 

c16 = np.complex128
plt.rc('font', family='serif')

Code adapted from the following sources
- Jonny Hyman's [implimentation](https://github.com/jonnyhyman/QuantumWaves)
- Original [implementation](https://github.com/Azercoco/Python-2D-Simulation-of-Schrodinger-Equation)

# Field

In [2]:
class Field:
    
    def __init__(self):
        self.potential_expr = None
        self.obstacle_expr = None

    def setPotential(self, expr):
        self.potential_expr = expr
        self.test_pot_expr()
        
    def setObstacle(self, expr):
        self.obstacle_expr = expr
        self.test_obs_expr()

    def test_pot_expr(self):
        # required for eval()
        x = 0
        y = 0
        try:
            a = eval(self.potential_expr)
        except:
            print(self.potential_expr)
            print('Potential calculation error: set to 0 by default')
            self.potential_expr = '0'
            
    def test_obs_expr(self):
        # required for eval()
        x = 0
        y = 0
        try:
            a = eval(self.obstacle_expr)
        except:
            print('Error setting obstacle: Set to False by default')
            self.obstacle_expr = 'False'
            
    def isObstacle(self, x, y):
        a = False
        try:
            a = eval(self.obstacle_expr)
        except:
            print(f'Invalid obstacle: {self.obstacle_expr}')
        return a
    
    def getPotential(self, x, y):
        a = 0 + 0j
        try:
            a = eval(self.potential_expr)
        except:
            print(f'Invalid potential: {self.potential_expr}')
        return a
    
    
    
def apply_obstacle(MM, N, meshX, meshY):
    for i in range(N):
        for j in range(N):
            if isObstacle(meshX[i][j], meshY[i][j]):
                MM[i][j] = 0 + 0j
    return MM


def getAdjPos(x, y, N):
    res = []
    res.append((x-1,y))
    res.append((x+1,y))
    res.append((x, y - 1))
    res.append((x,y+1))
    res.append((x - 1,y+1))
    res.append((x - 1,y-1))
    res.append((x + 1,y+1))
    res.append((x+1, y+1))
    return res


def colorize(z):
    r = np.abs(z)
    arg = np.angle(z) 

    h = (arg + np.pi)  / (2 * np.pi) + 0.5
    l = 1.0 - 1.0/(1.0 + 2*r**1.2)
    s = 0.8

    c = np.vectorize(hls_to_rgb) (h,l,s) # --> tuple
    c = np.array(c)  # -->  array of (3,n,m) shape, but need (n,m,3)
    c = c.swapaxes(0,2) 
    c = c.swapaxes(0,1) 
    return c

# Utils

In [3]:
do_parralel = False

@njit(cache=True, parallel=do_parralel)
def x_concatenate(MM, N):
    result = np.zeros((N*N),dtype=c16)
    for j in prange(N):
        for i in prange(N):
            index = i + N*j
            result[index] = (MM[i][j])
    return result


@njit(cache=True, parallel=do_parralel)
def x_deconcatenate(vector, N):
    result = np.zeros((N, N), dtype=c16)
    for j in prange(N):
        for i in prange(N):
            result[i][j] = vector[N*j + i]
    return result


@njit(cache=True, parallel=do_parralel)
def y_concatenate(MM, N):
    result = np.zeros((N*N),dtype=c16)
    for i in prange(N):
        for j in prange(N):
            index = j + N*i
            result[index] = (MM[i][j])
    return result


@njit(cache=True, parallel=do_parralel)
def y_deconcatenate(vector, N):
    result = np.zeros((N, N), dtype=c16)
    for i in prange(N):
        for j in prange(N):
            result[i][j] = vector[N*i + j]
    return result


@njit(cache=True, parallel=do_parralel)
def dx_square(MM, N, Δx):
    result = np.zeros((N, N), dtype=c16)
    for j in prange(N):
        result[0][j] = MM[1][j] - 2*MM[0][j]
        for i in prange(1, N-1):
            result[i][j] = MM[i+1][j] + MM[i-1][j] - 2*MM[i][j]
        result[N-1][j] = MM[N-2][j] - 2*MM[N-1][j]
    return result / (Δx**2)


@njit(cache=True, parallel=do_parralel)
def dy_square(MM, N, Δx):
    result = np.zeros((N, N), dtype=c16)
    for j in prange(N):
        result[j][0] = MM[j][1] - 2*MM[j][0]
        for i in prange(1, N-1):
            result[j][i] = MM[j][i+1] + MM[j][i-1] - 2*MM[j][i]
        result[j][N-1] = MM[j][N-2] - 2*MM[j][N-1]
    return result / (Δx**2)


@njit(cache=True)  
def integrate(MM, N, Δx):
    S = 0
    air = Δx*Δx/2
    for i in prange(N-1):
        for j in range(N-1):
            AA, AB, BA, BB = MM[i][j], MM[i][j+1], MM[i+1][j], MM[i+1][j+1]
            S += air*(AA+AB+BA)/3
            S += air*(BB+AB+BA)/3
    return S

# Main

In [4]:
N = 512
SIZE = 10
Δt = 0.001
Δx = SIZE/N
# V = "5*x**2 + 5*y**2" # 2D harmonic oscillator potential
V = "False"


x0 =  [-2.5]
y0 =  [ 0.0] 
k_x = [ 9.0] 
k_y = [ 0.0] 
a_x = [ 0.8] 
a_y = [ 0.8]


#Potential as a function of x and y
field = Field()
field.setPotential(V) # Ex: x**2+y**2"
potential_boudnary = []


#Obstacle: boolean expression in fct of x and y (set to False if you do not want an obstacle)
obstacles = "(x > 0.5 and x < 1 and not ((y > 0.1 and y < 0.6) or (y < -0.1 and y > -0.6)))"
# obstacles = "False"
field.setObstacle(obstacles)    
wall_potential = 1e10

In [ ]:
######## Create points at all xy coordinates in meshgrid ######## 
x_axis = np.linspace(-SIZE/2, SIZE/2, N)
y_axis = np.linspace(-SIZE/2, SIZE/2, N)
X, Y = np.meshgrid(x_axis, y_axis)


######## Initialize Wavepackets ######## 
n = 0
phase = np.exp( 1j*(X*k_x[n] + Y*k_y[n]))
px = np.exp( - ((x0[n] - X)**2)/(4*a_x[n]**2))
py = np.exp( - ((y0[n] - Y)**2)/(4*a_y[n]**2))

Ψ = phase*px*py
norm = np.sqrt(integrate(np.abs(Ψ)**2, N, Δx))
Ψ = Ψ/norm

for n in range(1,len(x0)):
    phase = np.exp( 1j*(X*k_x[n] + Y*k_y[n]))
    px = np.exp( - ((x0[n] - X)**2)/(4*a_x[n]**2))
    py = np.exp( - ((y0[n] - Y)**2)/(4*a_y[n]**2))

    Ψn = phase*px*py
    norm = np.sqrt(integrate(np.abs(Ψn)**2, N, Δx))

    Ψ += Ψn/norm

NORM = np.sqrt(integrate(np.abs(Ψ)**2, N, Δx))
Ψ = Ψ/NORM

  
######## Create Potential ######## 
V_x = np.zeros(N*N, dtype='c16')
for j in range(N):
    for i in range(N):
        xx = i
        yy = N*j
        if field.isObstacle(x_axis[j], y_axis[i]):
            V_x[xx+yy] = wall_potential
        else:
            V_x[xx+yy] = field.getPotential(x_axis[j], y_axis[i])

V_y = np.zeros(N*N, dtype='c16')
for j in range(N):
    for i in range(N):
        xx = j*N
        yy = i
        if field.isObstacle(x_axis[i], y_axis[j]):
            V_y[xx+yy] = wall_potential
        else:
            V_y[xx+yy] = field.getPotential(x_axis[i], y_axis[j])

V_x_matrix = sp.sparse.diags([V_x], [0])
V_y_matrix = sp.sparse.diags([V_y], [0])


######## Create Hamiltonian ########
LAPLACE_MATRIX = sp.sparse.lil_matrix(-2*sp.sparse.identity(N*N))
for i in range(N):
    for j in range(N-1):
        k = i*N + j
        LAPLACE_MATRIX[k,k+1] = 1
        LAPLACE_MATRIX[k+1,k] = 1

LAPLACE_MATRIX = LAPLACE_MATRIX/(Δx ** 2)

HX = (1*sp.sparse.identity(N*N) - 1j*(Δt/2)*(LAPLACE_MATRIX - V_x_matrix))
HX = csr_matrix(HX)

HY = (1*sp.sparse.identity(N*N) - 1j*(Δt/2)*(LAPLACE_MATRIX - V_y_matrix))
HY = csr_matrix(HY)


######## Place Obstacles ########  
for i in range(0, N):
    for j in range(0, N):
        if field.isObstacle(x_axis[j], y_axis[i]):
            adj = getAdjPos(i, j, N)
            for xx, yy in adj:
                coord_check = xx >= 0 and yy >= 0 and xx < N and yy <N 
                if coord_check and not field.isObstacle(x_axis[yy], y_axis[xx]):
                    potential_boudnary.append((i, j))

In [ ]:
def evolve(Ψ):

    vector_wrt_x = x_concatenate(Ψ, N)
    vector_deriv_y_wrt_x = x_concatenate(dy_square(Ψ, N, Δx), N)
    U_wrt_x = vector_wrt_x + (1j*Δt/2)*(vector_deriv_y_wrt_x - V_x*vector_wrt_x)
    U_wrt_x_plus = scipy.sparse.linalg.spsolve(HX, U_wrt_x)
    Ψ = x_deconcatenate(U_wrt_x_plus, N)


    vector_wrt_y = y_concatenate(Ψ, N)
    vector_deriv_x_wrt_y = y_concatenate(dx_square(Ψ, N, Δx), N)
    U_wrt_y = vector_wrt_y  + (1j*Δt/2)*(vector_deriv_x_wrt_y - V_y *vector_wrt_y)
    U_wrt_y_plus = scipy.sparse.linalg.spsolve(HY, U_wrt_y)
    Ψ = y_deconcatenate(U_wrt_y_plus, N)   

    return Ψ

In [ ]:
N_iter = 1000
Ψ_arr = np.zeros((N_iter, N, N), dtype='c16')

start = time()
for i in range(N_iter):
    Ψ = evolve(Ψ)
    Ψ_arr[i] = Ψ
    NORM = np.sqrt(integrate(np.abs(Ψ)**2, N, Δx))
    clear_output(wait=True)
    print("Iteration {} / {}".format(i, N_iter))
    print('Function norm : {0:.9f} '.format(NORM))
    
end = time()
print("{:.1f} minutes".format((end-start)/60))

# 2D Visualization

Quick animation of the probability amplitude $ |\psi(x,y,t)|^{2} $

In [ ]:
cmap = plt.cm.RdBu

for i in range(N_iter):    
    rgb_map = cmap(np.abs(Ψ_arr[i])**2)
    rgb_map = rgb_map[:, :, :3]
    for i, j in potential_boudnary:
        rgb_map[i][j] = 1, 1, 1
    img = Image.fromarray(np.uint8(rgb_map * 255))
    clear_output(wait=True)
    display(img)

In [ ]:
def get_rgb_maps(Ψ_arr, i):
    real_rgb_map = colorize(Ψ_arr[i,:,:].real)
    imag_rgb_map = colorize(Ψ_arr[i,:,:].imag)
    prob_rgb_map = colorize(Ψ_arr[i,:,:])

    for i, j in potential_boudnary:
        real_rgb_map[i][j] = 1, 1, 1
        imag_rgb_map[i][j] = 1, 1, 1
        prob_rgb_map[i][j] = 1, 1, 1
        
    return real_rgb_map, imag_rgb_map, prob_rgb_map


def plot_iteration_2D(i):
    
    real_rgb_map, imag_rgb_map, prob_rgb_map = get_rgb_maps(Ψ_arr, i)
        
    fig = plt.figure(figsize=(30,20), dpi=100, constrained_layout=True)
    gs = fig.add_gridspec(2, 3)
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(real_rgb_map)
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    ax1.set_title(r" $\psi(x,y)$ Real ", fontsize=40)
    plt.axis('off')
    
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.imshow(imag_rgb_map)
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    ax2.set_title(r" $\psi(x,y)$ Imaginary ", fontsize=40)
    plt.axis('off')
    
    ax3 = fig.add_subplot(gs[0:, 1:])
    ax3.imshow(prob_rgb_map)
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.set_title(r"$| \psi(x,y) |^{2}$ ", fontsize=40)
    plt.axis('off')
    
    plt.show()

In [ ]:
plot_iteration_2D(i=150)

# 3D Visualization

In [ ]:
def plot_iteration_3D(i, shade=False):
    
    real_rgb_map, imag_rgb_map, prob_rgb_map = get_rgb_maps(Ψ_arr, i)
    
    fig = plt.figure(figsize=(30,20), dpi=200, constrained_layout=True)
    gs = fig.add_gridspec(2, 3)
    
    ax1 = fig.add_subplot(gs[0, 0], projection='3d')
    ax1.plot_surface(X, Y, Ψ_arr[i,:,:].real, 
                     facecolors=real_rgb_map, rstride=1, cstride=1, shade=shade) 
    ax1.view_init(20,230)
    ax1.set_zlim([-0.5,0.5])
    ax1.set_xlim(-5,5)
    ax1.set_ylim(-5,5)
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    ax1.set_title(r" $\psi(x,y)$ Real ", fontsize=35)
    plt.axis('off')
    
    ax2 = fig.add_subplot(gs[1, 0], projection='3d')
    ax2.plot_surface(X, Y, Ψ_arr[i,:,:].imag, 
                     facecolors=imag_rgb_map, rstride=1, cstride=1, shade=shade) 
    ax2.view_init(20,230)
    ax2.set_zlim([-0.5,0.5])
    ax2.set_xlim(-5,5)
    ax2.set_ylim(-5,5)
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    ax2.set_title(r" $\psi(x,y)$ Imaginary ", fontsize=35)
    plt.axis('off')
     
    ax3 = fig.add_subplot(gs[0:, 1:], projection='3d')
    ax3.plot_surface(X, Y, np.abs(Ψ_arr[i,:,:])**2, 
                     facecolors=prob_rgb_map, rstride=1, cstride=1, shade=shade) 
    ax3.view_init(20,230)
    ax3.set_zlim([0,0.8])
    ax3.set_xlim(-5,5)
    ax3.set_ylim(-5,5)
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.set_title(r" $ | \psi(x,y) |^{2} $ ", fontsize=35)
    plt.axis('off')
    
    plt.show()

In [ ]:
start = time() 
plot_iteration_3D(i=0, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

In [ ]:
start = time() 
plot_iteration_3D(i=90, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

In [ ]:
start = time() 
plot_iteration_3D(i=150, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

In [ ]:
start = time() 
plot_iteration_3D(i=250, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

In [ ]:
start = time() 
plot_iteration_3D(i=500, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

In [ ]:
start = time() 
plot_iteration_3D(i=750, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

In [ ]:
start = time() 
plot_iteration_3D(i=999, shade=False)
end = time()
print("\n{:.0f} seconds".format(end-start))

# 2D Animations

In [ ]:
real_rgb_map, imag_rgb_map, prob_rgb_map = get_rgb_maps(Ψ_arr, i=0)

def update_plot(frame_number, Ψ_array, plots):
    real_rgb_map, imag_rgb_map, prob_rgb_map = get_rgb_maps(Ψ_array, i=frame_number)
    
    plots[0].remove()
    plots[1].remove()
    plots[2].remove()
    
    plots[0] = ax1.imshow(real_rgb_map)
    plots[1] = ax2.imshow(imag_rgb_map)
    plots[2] = ax3.imshow(prob_rgb_map)
    
plots = []
fig = plt.figure(figsize=(30,20), dpi=100, constrained_layout=True)
gs = fig.add_gridspec(2, 3)

ax1 = fig.add_subplot(gs[0, 0])
plots.append(ax1.imshow(real_rgb_map))
ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
ax1.set_title(r" $\psi(x,y)$ Real ", fontsize=40)
plt.axis('off')

ax2 = fig.add_subplot(gs[1, 0])
plots.append(ax2.imshow(imag_rgb_map))
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)
ax2.set_title(r" $\psi(x,y)$ Imaginary ", fontsize=40)
plt.axis('off')

ax3 = fig.add_subplot(gs[0:, 1:])
plots.append(ax3.imshow(prob_rgb_map))
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
ax3.set_title(r"$| \psi(x,y) |^{2}$ ", fontsize=40)
plt.axis('off')

start = time() 

anim = animation.FuncAnimation(fig, update_plot, frames=N_iter, fargs=(Ψ_arr, plots), interval=10)
anim.save('src/schrodinger_2D_512.mp4', fps=30)
plt.close()

end = time()
print("\n{:.0f} minutes".format((end-start)/60))

# 3D Animations

Matploblib isn't made for these types of animations, so it takes a super long time. For a 1000 frame animation on a 512x512
grid, it can take over a day to complete. 

In [ ]:
psi = 230
real_rgb_map, imag_rgb_map, prob_rgb_map = get_rgb_maps(Ψ_arr, i=0)

def update_plot(frame_number, Ψ_array, plots):
    global psi
    psi = psi + 0.08
    
    real_rgb_map, imag_rgb_map, prob_rgb_map = get_rgb_maps(Ψ_array, frame_number)
    
    plots[0].remove()
    plots[1].remove()
    plots[2].remove()
    
    plots[0] = ax1.plot_surface(X, Y, Ψ_array[frame_number,:,:].real, 
                              facecolors=real_rgb_map, rstride=1, cstride=1, shade=False)
    plots[1] = ax2.plot_surface(X, Y, Ψ_array[frame_number,:,:].imag, 
                              facecolors=imag_rgb_map, rstride=1, cstride=1, shade=False)
    plots[2] = ax3.plot_surface(X, Y, np.abs(Ψ_array[frame_number,:,:])**2, 
                              facecolors=prob_rgb_map, rstride=1, cstride=1, shade=False)
    
    ax1.view_init(20,psi)
    ax2.view_init(20,psi)
    ax3.view_init(20,psi)
    
    
plots = []
fig = plt.figure(figsize=(30,20), dpi=100, constrained_layout=True)
gs = fig.add_gridspec(2, 3)

ax1 = fig.add_subplot(gs[0, 0], projection='3d')
plots.append(ax1.plot_surface(X, Y, Ψ_arr[0,:,:].real, 
                              facecolors=real_rgb_map, rstride=1, cstride=1, shade=False))
ax1.view_init(20,230)
ax1.set_zlim([-0.5,0.5])
ax1.set_xlim(-5,5)
ax1.set_ylim(-5,5)
ax1.get_xaxis().set_visible(False) 
ax1.get_yaxis().set_visible(False)
ax1.set_title(r" $\psi(x,y)$ Real ", fontsize=40)
plt.axis('off')

ax2 = fig.add_subplot(gs[1, 0], projection='3d')
plots.append(ax2.plot_surface(X, Y, Ψ_arr[0,:,:].imag, 
                              facecolors=imag_rgb_map, rstride=1, cstride=1, shade=False))
ax2.view_init(20,230)
ax2.set_zlim([-0.5,0.5])
ax2.set_xlim(-5,5)
ax2.set_ylim(-5,5)
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)
ax2.set_title(r" $\psi(x,y)$ Imaginary ", fontsize=40)
plt.axis('off')

ax3 = fig.add_subplot(gs[0:, 1:], projection='3d')
plots.append(ax3.plot_surface(X, Y, np.abs(Ψ_arr[0,:,:])**2, 
                              facecolors=prob_rgb_map, rstride=1, cstride=1, shade=False))
ax3.view_init(20,230)
ax3.set_zlim([0,0.8])
ax3.set_xlim(-5,5)
ax3.set_ylim(-5,5)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
ax3.set_title(r"$| \psi(x,y) |^{2}$ ", fontsize=40)
plt.axis('off')

start = time() 

anim = animation.FuncAnimation(fig, update_plot, frames=N_iter, fargs=(Ψ_arr, plots), interval=10)
anim.save('src/schrodinger_3D_512.mp4', fps=30)
plt.close()

end = time()
print("\n{:.0f} minutes".format((end-start)/60))